In [130]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

#base e dados da base
columns = ["loc", "v(g)", "ev(g)","iv(g)", "n", "v", "l", "d","i", "e", "b", "t", "lOCode",\
                                      "loComment", "lOBlank", "locCodeAndComment", "uniq_Op", "uniq_Opnd", \
                                      "total_Op", "total_Opnd", "branchCount", "defects"]
df = pd.read_csv("../base2.csv", names=columns)
size = df.shape[1]
# Samples for training
tam = (size - 1)
feature_data = df.iloc[:, :tam]
label_data = df.iloc[:, -1]

x = feature_data.values #returns a numpy array
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data = pd.DataFrame(x_scaled, columns=df.columns[:tam])

In [125]:
import numpy.linalg as la 
import numpy as np
cov = data.cov()
eigenvalues, eigenvectors = la.eig(cov)
# Get higher eigenvectors
k = 10
eigenvalues_indices = np.argsort(eigenvalues)[::-1]
a = eigenvalues_indices[:k]
b = eigenvalues_indices[-1:]
eigenvalues_indices = np.concatenate([a,b])

In [126]:
np.dot(eigenvectors[5], eigenvectors[4])

1.1657341758564144e-14

In [129]:
print(cov.shape)
print(eigenvectors[eigenvalues_indices].shape)
new_data = np.matrix(eigenvectors[eigenvalues_indices])*np.matrix(x_scaled.T)
df_new = pd.DataFrame(new_data.T, columns=df.columns[eigenvalues_indices])
#df_new.loc.dtype
#auto_valores/np.sum(auto_valores)

(21, 21)
(11, 21)


In [136]:
# Training model
model = KNeighborsClassifier(n_neighbors=1)

# Slip data mode
cv = StratifiedKFold(n_splits=5)

#training data
tam = df_new.shape[1] - 1
feature_new_data = df_new.iloc[:, :tam]

# Executing the train and test process
scores = cross_val_score(model, feature_new_data, label_data, cv=cv)

print("Model's acuracy")
print(scores)

Model's acuracy
[0.86363636 0.69318182 0.67816092 0.86046512 0.86046512]
